In [28]:
!python3 --version
#!python --version
#!jupyter --version

import inspect

from algebra import *
from gauss_iterative import *
from gauss_recursive import *
from example import *

Python 3.11.2


# Gaussian Elimination in plain python (no deps)

## Motivation
_C_

Gaussian algorithm is constantly performed in scientific computing of matrix.
It is the basis of investigating many properties of matrix, like rank, inverse, kernel and of course the linear equation system. 

In [29]:
show(Demo_Matrix)

4	1	2	3
2	0.5	1	-2
0	3	0	2
4	-2	-1	3



### Goal
Translate the often human oriented algorithm instructions into proper tasks working on a computer.

We have to find representations in code for concepts like "cross the first column off mentally" or
"swap the row with a *suitable* row". 

We need to judge whether it makes sense to stick to the recipe (global vs. local vision) or if there are some (heuristic) shortcuts for the computer.

![](gauss1.jpeg)
_Source: Page 60 of the textbook of Fraleigh and Beauregard on Linear Algebra (3rd edition, 1995)._


## Implementation
_N_
### Datatypes for type hints
```python
from fractions import Fraction
F = Fraction | float | int 
R = list[F]
M = list[R]    # 0-based indexing
```

### Algebra on Matrix
```python
# Scalar Multiplication
def scalar_mult(M1: M, k: F) -> M
# Addition
def add(M1: M, M2: M) -> M
# Multiplication
def mult(M1: M, M2: M) -> M
# Take the cth-column
def column(M1: M, c: int) -> R
```

### Actions and their Elementary Matrices
```python
# Identity Matrix
def I(n: int) -> M
# Swap two rows
def S(n: int, r1: int, r2: int) -> M
# Multiply a row a times
def M(n: int, r1: int, a: F) -> M
# Add a times of r2 into r1
def A(n: int, r1: int, r2: int, a: F) -> M
```

### Properties
```python
def is_nullrow(row: R) -> bool:

def is_identity_matrix(m: M) -> bool:

```
### Helpers
```python
def show(m: M):
def show_indent(m: M, indent: int):

class StepByStep:
     """A class containing matrix and a stack of elementary operations,
     applying them one by one"""
```

### Basic algebra function in use

In [30]:
simpleM = [ [1,2,3], [2,0,1], [1,2,5] ]

show(simpleM)

## Scale the matrix by 10:
show(scalar_mult(simpleM, 10))

## Scale row0 by 3
show(mult(M(3, 0, 3), simpleM))

## Project first column:
print(column(simpleM, 0))

1	2	3
2	0	1
1	2	5

10	20	30
20	0	10
10	20	50

3	6	9
2	0	1
1	2	5

[1, 2, 1]


## Demos
Let's first show some examples.

In [31]:
show(Real_Matrix)

3.50	-2.00	1.00
-3.10	5.60	0.00
4.70	0.00	-1.30



In [32]:
Real_echelon, _, Elementary_trace = gauss_algorithm_iterative(Real_Matrix, True)
show(Real_echelon)

3.50	-2.00	1.00
0.00	3.83	0.89
0.00	0.00	-3.26



In [33]:
steps = StepByStep(Real_Matrix, Elementary_trace)

In [34]:
while True:
    try:
        next(steps)
        input()
    except:
        print("This is the end of the algorithm!")
        break

3.50	-2.00	1.00
0.00	3.83	0.89
4.70	0.00	-1.30



 1


3.50	-2.00	1.00
0.00	3.83	0.89
0.00	2.69	-2.64



 12


3.50	-2.00	1.00
0.00	3.83	0.89
0.00	0.00	-3.26



 89


This is the end of the algorithm!


In [35]:
show(Rational_Matrix)

2	1	0	0
3	5	-1	-3
0	3	-1	0
1	4	0	-2



In [36]:
Rational_reduced_echelon, rank, Elementary_trace = normalize(Rational_Matrix, True)
print("Rank of this matrix is:", rank)
show(Rational_reduced_echelon)

Rank of this matrix is: 4
1	0	0	0
0	1	0	0
0	0	1	0
0	0	0	1



In [37]:
steps = StepByStep(Rational_Matrix, Elementary_trace)

In [39]:
while True:
    try:
        next(steps)
        input()
    except:
        print("This is the end of the algorithm!")
        break

This is the end of the algorithm!


### Computing the inverse
_C_

We can compute the inverse of an invertible matrix by simply multipling all elementary matrices together.

In [40]:
# But be careful that the matrix multiplications don't commute!
Elementary_trace.reverse()
Inverse = reduce(mult, Elementary_trace, I(rank))
show(Inverse)
show(mult(Rational_Matrix, Inverse))

8/19	2/19	-2/19	-3/19
3/19	-4/19	4/19	6/19
9/19	-12/19	-7/19	18/19
10/19	-7/19	7/19	1/19

1	0	0	0
0	1	0	0
0	0	1	0
0	0	0	1



## Details of the Implementation
_C_

### Iterative Version

Perform Gauss algorithm (iterative)

```python
def gauss_algorithm_iterative(m: M, is_traced=False) -> tuple[M, int, list[M]]
```

1. Get most-left column with non-zero values, find best row for first column, otherwise ignore this column by increasing now_column
2. If the top-row value is zero, then swap now_row with last non-zero row (or put to bottom using nullrow_cnt)
3. Make zeroes below the pivot (by adding the respective inverse multiple)
4. Perform 1-3 with remaining rows.

It returns the reduced matrix(in echelon form), the rank and the trace of operations

```python
    nullrow_cnt = 0
    now_row = 0
    now_column = 0
    row_dim = len(m)
    trace = []
    while now_row < row_dim - nullrow_cnt:
        (pivot_index, pivot) = get_pivot(m[now_row])

        if pivot_index == None:  # it's a nullrow
            swap = S(row_dim, now_row, row_dim - 1 - nullrow_cnt)
            trace.append(swap)
            m = mult(swap, m)
            nullrow_cnt += 1
        else:
            if pivot_index > now_column:  # there's still better pivot column at left
                better_candidate = find_better_candidate(
                    m, now_column, pivot_index, now_row
                )
                swap = S(row_dim, now_row, better_candidate)
                trace.append(swap)
                m = mult(swap, m)
                (pivot_index, pivot) = get_pivot(
                    m[now_row]
                )  # after swapping, we must get the pivot again
            col = column(m, pivot_index)
            scalar = list(map(lambda c: c / pivot, col))
            for k in range(now_row + 1, row_dim - nullrow_cnt):
                if (
                    -scalar[k] < 0 or -scalar[k] > 0
                ):  # if already 0, we don't need to do anything
                    addition = A(row_dim, k, now_row, -scalar[k])
                    trace.append(addition)
                    m = mult(addition, m)
            now_row += 1
            now_column += 1
    if is_traced:
        return (m, row_dim - nullrow_cnt, trace)
    else:
        return (m, row_dim - nullrow_cnt, [])
```

Let's check how `get_pivot` works:

In [41]:

R1 = [3, 4, 0]
print(get_pivot(R1), '\t is (index, value) of the pivot of row ', R1)

R2 = [0, 0, 0]
print(get_pivot(R2), '\t is (index, value) of the pivot or row ', R2)

(0, 3) 	 is (index, value) of the pivot of row  [3, 4, 0]
(None, None) 	 is (index, value) of the pivot or row  [0, 0, 0]


And also `find_better_candidate`:

In [42]:
M1 = [[0, 0, 5], [0, 1, 0], [2, 4, 0]]
show(M1)

0	0	5
0	1	0
2	4	0



In [43]:
print(find_better_candidate(M1, 0, 2, 0))

2


Normalize makes the matrix into a reduced echelon matrix.

```python
def normalize(m: M, is_traced=False) -> tuple[M, int, list[M]]
```

It applies the gauss algorithm first, then it goes from bottom(non-zero row) to top with following steps:

1. Normalize the pivot element of this row into 1
2. Make the element above the pivot zero by adding the respective additive inverse.
3. Repeat 1-2 on the above row till the first row.

It returns a tuple with reduced matrix, the rank of the matrix and trace of operations

```python
    row_dim = len(m)
    (m, rank, trace) = gauss_algorithm_iterative(m, is_traced)
    pivots = get_pivots(m) #get all pivots to make it faster later
    for k in range(rank):
        mul = M(row_dim, rank - k - 1, 1 / pivots[rank - k - 1][1]) #normalizing
        if is_traced:
            trace.append(mul)
        m = mult(mul, m)
        col_index = pivots[rank - k - 1][0] #the column above this pivot should be cleared
        for r in range(rank - k - 1):
            if -m[r][col_index] < 0 or -m[r][col_index] > 0:
                addition = A(row_dim, r, rank - k - 1, -m[r][col_index]) #make the element zero
                if is_traced:
                    trace.append(addition)
                m = mult(addition, m)
    if is_traced:
        return (m, rank, trace)
    else:
        return (m, rank, [])
```

## Recursive implementation
_N_

![](gauss_recursive.png)

### Recursive Algorithm

```python
trace = True


def find_pivot_row_index(column: R) -> int:
    """Assuming `column` contains a non-zero value, find it's (row) index"""
    for i in range(0, len(column)):
        if column[i] != 0:
            return i
        else:
            i += 1
            continue


def gauss_rec(
    m: M, nowrow: int, nowcol: int, n_rows: int, n_cols: int, stack, depth: int
) -> tuple[M, list[M]]:
    """
    Recursive implementation of gauss elimination.

    1. Establish a useful toprow by finding left-most pivot row. Skip zero columns and swap rows if necessary.
    1.1 As long as zero-column, increment nowcol
    1.2 Current nowcol is guaranteed to have a non-zero entry in some row. (as it's not a zero column)
    2. Create zeroes below the pivot
    3. Solve recursively for m with `nowrow - 1` and `nowcol - 1`
    
    Return current matrix and stack (at any point) once running out of rows or cols.

    Prints the transformed matrices for each step, with a natural language description and
    returns the accumulated list of elementary_matrices.

    """

    indentation = "\t" * depth

    if trace: print(f"\n{indentation}Working on matrix of size {n_rows - nowrow} x {n_cols - nowcol}")

    # Base case of recursion
    if nowrow == n_rows - 1 or nowcol == n_cols - 1:
        return m, stack

    
    # 1. Skip any zero columns

    while is_nullrow(column(m[nowrow:], nowcol)):
        if trace: print(f"\n{indentation}Skipping at least one zero-column...")
        nowcol += 1
        if nowcol == n_cols - 1:  # Another, more hidden base case
            return m, stack

    if trace: print(f"\n{indentation}-- Establish a useful toprow --")

    # 2. Establish a useful toprow, swap if necessary. column(m, nowcol) is guaranteed to have a pivotrow
    
    pivot = m[nowrow][nowcol]
    if pivot == 0:
        better_row = find_pivot_row_index(column(m[nowrow + 1:], nowcol)) + nowrow + 1
        elem_swap = S(n_rows, nowrow, better_row)
        if trace:
            stack.append(elem_swap)
        m = mult(elem_swap, m)
        pivot = m[nowrow][nowcol] # update pivot to reflect row swap
        assert pivot != 0
        if trace:
            print(
                    f"\n{indentation}Swapped row {nowrow + 1} with good pivot row {better_row + 1}\n"
            )
            show_indent(m, depth)
    elif trace:
        print(f"\n{indentation}No need to swap rows. Current pivot is fine\n")

    

    # TODO: normalize toprow by dividing by it's pivot simplifying the computation of inv_scalar.

    if trace:
        print(f"\n{indentation}-- Create zeroes below the pivot --")
        
    # 3. Create zeroes below the pivot
    
    for rowindex in range(nowrow + 1, n_rows):
        if m[rowindex][nowcol] == 0: continue  # entry below pivot is already 0
        else:
            inv_scalar = -(m[rowindex][nowcol] / pivot)
            elem_add = A(n_rows, rowindex, nowrow, inv_scalar)
        if trace: stack.append(elem_add)
        m = mult(elem_add, m)
        if trace: print(
                f"\n{indentation}Created 0 below pivot in row {rowindex + 1} by adding {inv_scalar} * row {nowrow + 1} to it.\n"
            )
            show_indent(m, depth)

    return gauss_rec(m, nowrow + 1, nowcol + 1, n_rows, n_cols, stack, depth + 1)


def gauss_rec_go(m: M):
    """run recursive gauss_split with initial values"""
    n_rows = len(m)
    n_cols = len(m[0])
    if n_rows == 1 and n_cols == 1:
        return m
    if m == [[0] * n_cols] * n_rows:
        return m
    print("Bring the following matrix to row_echelon form:\n")
    show(m)
    return gauss_rec(m, 0, 0, n_rows, n_cols, [], 0)

```

In [44]:
matrix_row_echelon, stack = gauss_rec_go(Demo_Matrix)   # with inline output

steps = StepByStep(Demo_Matrix, stack)

Bring the following matrix to row_echelon form:

4	1	2	3
2	0.5	1	-2
0	3	0	2
4	-2	-1	3


Working on matrix of size 4 x 4

-- Establish a useful toprow --

No need to swap rows. Current pivot is fine


-- Create zeroes below the pivot --

Created 0 below pivot in row 2 by adding -0.5 * row 1 to it.

4	1.0	2	3
0.0	0.0	0.0	-3.5
0	3.0	0	2
4	-2.0	-1	3

Created 0 below pivot in row 4 by adding -1.0 * row 1 to it.

4.00	1.00	2.00	3.00
0.00	0.00	0.00	-3.50
0.00	3.00	0.00	2.00
0.00	-3.00	-3.00	0.00

	Working on matrix of size 3 x 3

	-- Establish a useful toprow --

	Swapped row 2 with good pivot row 3

	4.00	1.00	2.00	3.00
	0.00	3.00	0.00	2.00
	0.00	0.00	0.00	-3.50
	0.00	-3.00	-3.00	0.00

	-- Create zeroes below the pivot --

	Created 0 below pivot in row 4 by adding 1.0 * row 2 to it.

	4.00	1.00	2.00	3.00
	0.00	3.00	0.00	2.00
	0.00	0.00	0.00	-3.50
	0.00	0.00	-3.00	2.00

		Working on matrix of size 2 x 2

		-- Establish a useful toprow --

		Swapped row 3 with good pivot row 4

		4.00	1.00	2.0

In [45]:
next(steps)

4	1.0	2	3
0.0	0.0	0.0	-3.5
0	3.0	0	2
4	-2.0	-1	3



[[4, 1.0, 2, 3], [0.0, 0.0, 0.0, -3.5], [0, 3.0, 0, 2], [4, -2.0, -1, 3]]

### Think locally, act global
_N_

It's not trivial to think about the local and global state at the same time.
We have access to the full matrix at each recursion depth, because we need
to perform the elementary-matrices to the whole matrix.

Although it's also possible to pass smaller matrices down the recursion.
but it's more complicated to merge all those submatrices back together.

So this mixture of local/global view makes it harder to choose the correct
variables to derive our indices from and also has a lot of off-by-one potential.


## Improvements / Refactoring
_C_

- The gauss function is quite big and deeply nested. It would be nice to have separate functions matching the steps in the algorithm.
- Implement different approaches, run performance tests and analyze. Maybe some heuristics can help to speed up unlucky cases (e.g. pivots all the way to the right)
- What about correctness?
  - e.g. numerical issues (convert int to fractions?)
- Proper testing of edge cases.
- Check properties and assert they hold within the algorithm. (like invariants)
- Represent `Matrix` as a class with properties and functions; find more appropriate model than `list[list[F]]`
- The bottleneck of this algorithm is the naive matrix multiplication (in a production environment it will be replaced by robust libraries and GPU acceleration)
- Swapping lists instead of swapping via elementary_swap_matrix could be cheaper.

### numerical issues of unreal reals

In [24]:
# yes
print(0.2 + 0.2 == 0.4)

# but
print(0.2 + 0.1 == 0.3)
print(0.2 - 0.2 == (((0.3 - 0.1) - 0.1) - 0.1))

True
False
False


In [46]:
# See how long it takes!
gauss_algorithm_iterative(Big_Matrix)

KeyboardInterrupt: 

## Appendix, more code samples


### Functional Paradigm / Category Theory

- `map` applies a function on `a` to a list of `a`s:
```map(increment, [1,2,3]) -> [increment(2), increment(3), increment(4)] -> [2,3,4]```
- `reduce` / `fold` combines two elements of a list and accumulates the result
  `fold(add, [1,2,3]) -> add(add(1,2),3)` -> 6
- `zip` two lists, pairs them up: `zip([1,2], [10,20]) -> [(1,10), (2,20)]`
- `filter` takes a property function and a list and returns only those elements that match the property.


### inverting, asserting identity, non-invertible

In [47]:
simpleM = [ [1,2,3], [2,0,1], [1,2,5] ]

show(simpleM)
show(mult(I(len(simpleM)), simpleM))

show(inverse(simpleM))
show(mult(simpleM, inverse(simpleM))) # == id

assert is_identity_matrix(mult(simpleM, inverse(simpleM)))

1	2	3
2	0	1
1	2	5

1	2	3
2	0	1
1	2	5

0.25	0.50	-0.25
1.12	-0.25	-0.62
-0.50	0.00	0.50

1.00	0.00	0.00
0.00	1.00	0.00
0.00	0.00	1.00



In [48]:
show(I(len(simpleM)))

1	0	0
0	1	0
0	0	1



In [49]:
# not invertible
notinvM = [ [1,2,3], [0,0,1], [1,2,5]]

gauss_rec_go(notinvM)


try:
    show(inverse(notinvM))  # inverting fails
except:
    print("\n\n\nSorry, this failed. Could not invert that matrix\n")

Bring the following matrix to row_echelon form:

1	2	3
0	0	1
1	2	5


Working on matrix of size 3 x 3

-- Establish a useful toprow --

No need to swap rows. Current pivot is fine


-- Create zeroes below the pivot --

Created 0 below pivot in row 3 by adding -1.0 * row 1 to it.

1	2	3
0	0	1
0.0	0.0	2.0

	Working on matrix of size 2 x 2

	Skipping at least one zero-column...



Sorry, this failed. Could not invert that matrix

